In [1]:
# bertsum
!pip install bert-extractive-summarizer==0.4.2
# t5
!pip install transformers
!pip install SentencePiece

In [9]:
TEXTFILENAME = 'talk.txt'
int_max_length=5000
out_max_length=100

f = open('/{}'.format(TEXTFILENAME), mode='r', encoding='utf-8')
x = f.read()
f.close()

def speaker(x):
  talk_str = x.split("\n")
  talk_str
  total = []

  speaker_num=0
  for i in talk_str:
    try:
      if int(i[8])>speaker_num:
        speaker_num = int(i[8])
    except:
      pass
 
  num = 0
  speaker_talk = [[] for row in range(speaker_num)]
  for i in talk_str:
    if len(i)>30:
      try:
        num = int(i[8])-1
        j = i[11:].split(".")
        clean_list=[]
        for k in j:
          if len(k)>2:
            k = k.strip()
            total.append(k)
            clean_list.append(k)
        speaker_talk[num].extend(clean_list)
      except:
        pass
    else:
      pass
  return speaker_talk, speaker_num, total


def bertsum(out_max_length, speaker_talk, speaker_num, total):
  from summarizer import Summarizer

  f = open("speaker_summary_bertsum.txt", 'w')
  f.write("speaker : "+str(speaker_num)+"\n\n")
  for i in range(speaker_num):
    text = ". ".join(speaker_talk[i])
    f.write("speaker"+str(i+1)+" :\n")
    model = Summarizer('distilbert-base-uncased')
    output = model(text,max_length = out_max_length)
    f.write(output+"\n")
  out_text = ". ".join(total)
  total_output = model(out_text,max_length = out_max_length)
  f.write("\nsummary : "+total_output+"\n")
  f.close()

def t5(int_max_length, out_max_length, speaker_talk, speaker_num, total):
  
  int_max_length = int_max_length
  out_max_length = out_max_length

  f = open("speaker_summary_t5.txt", 'w')
  f.write("speaker : "+str(speaker_num)+"\n\n")

  for i in range(speaker_num):
    text = ".".join(speaker_talk[i])
    f.write("speaker"+str(i+1)+" :\n")
    output = t5_model(text,int_max_length,out_max_length)
    f.write(output+"\n")

  out_text = ".".join(total)
  output = t5_model(out_text,int_max_length,out_max_length)
  f.write("\nsummary : "+output+"\n")
  f.close()

def t5_model(text,int_max_length,out_max_length):
  import torch
  from transformers import T5Tokenizer,T5ForConditionalGeneration,T5Config

  model = T5ForConditionalGeneration.from_pretrained('t5-base')
  tokenizer = T5Tokenizer.from_pretrained('t5-base')
  device = torch.device('cpu')
  tokenized_text = tokenizer.encode(text, return_tensors="pt",max_length=int_max_length, truncation=True).to(device)
  summary_t5 = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=out_max_length,
                                    early_stopping=True)
  output = tokenizer.decode(summary_t5[0], skip_special_tokens=True)

  return output





In [10]:
speaker_talk, speaker_num, total = speaker(x)

In [7]:
# bertsum
bertsum(out_max_length, speaker_talk, speaker_num, total)

In [8]:
# t5
t5(int_max_length, out_max_length, speaker_talk, speaker_num, total)